In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import NoSuchElementException ,StaleElementReferenceException

In [2]:
url = 'https://www.cotodigital3.com.ar'

#this func takes the first url all with Beatiful Soup and the names, categories
def take_url():
    #taking the url
    r = requests.get(url)


    soup =BeautifulSoup(r.text,'html.parser')
    #selecting the tags
    classes=soup.find_all('ul',class_='sub_category')
    
    #looking at the href
    links= [i.find('a')['href'] for i in classes]

    #names of categories
    names=[i.a.text   for i in classes]
    #taking out the spaces and extra symbols
    names_categories = [i.replace(' (+)','').strip() for i in names]
                               
    return links ,names_categories
    


#this variable holds the function of categories and first links. This links have to iterate to another pages
first_step = take_url()

In [3]:
first_step[1]

['Golosinas',
 'Panaderia',
 'Snacks',
 'Cereales',
 'Endulzantes',
 'Aderezos y salsas',
 'Infusiones',
 'Conservas',
 'Encurtidos',
 'Harinas',
 'Mermeladas y dulces',
 'Salsas y puré de tomate',
 'Aceites y condimentos',
 'Alimento de bebés y niños',
 'Arroz y legumbres',
 'Especias',
 'Pasta seca, lista y rellenas',
 'Polvo para postres y reposteria',
 'Sopas, caldos, puré y saborizantes',
 'Rebozador y pan rallado',
 'Leche en polvo',
 'Suplementos dietarios',
 'Productos orgánicos',
 'Bebidas sin alcohol',
 'Bebidas con alcohol',
 'Lácteos',
 'Fiambres',
 'Quesos',
 'Carniceria',
 'Aves',
 'Pastas frescas y tapas',
 'Comidas elaboradas',
 'Frutas y verduras',
 'Pescaderia',
 'Huevos',
 'Helados y postres',
 'Comidas congeladas',
 'Hamburguesas y milanesas',
 'Vegetales congelados',
 'Papas congeladas / fritas',
 'Frutas congeladas',
 'Nuggets, patitas y bocaditos',
 'Pescaderia',
 'Lavado',
 'Papeles',
 'Insecticidas',
 'Calzado',
 'Accesorios de limpieza',
 'Desodorantes de ambi

In [4]:
first_step[0]

['/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-panaderia/_/N-s3bf1a',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-snacks/_/N-10kzbyj',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-cereales/_/N-ukd5id',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-endulzantes/_/N-1rtbab6',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-aderezos-y-salsas/_/N-rv0frc',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-infusiones/_/N-dw58vw',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-conservas/_/N-1t4efca',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-encurtidos/_/N-12rkdi1',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-harinas/_/N-842qrm',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-mermeladas-y-dulces/_/N-mj4aa8',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-salsas-y-pur%C3%A9-de-tomate/_/N-os1anu',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-aceites-y-condimentos/_/N-18r69ct',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-alimento-de-beb%C3

In [46]:
def take_product(categorias,url):



    r = requests.get(url)


    soup =BeautifulSoup(r.text,'html.parser')

    products = soup.find_all('li',class_='clearfix')

    
    count_product=0
    product_list=[]
    for product_description in products:

        #contador
        count_product=count_product+1
        #this is the count fraction to check the scrapped elements
        print(f'categoria : {categorias}')
        print(f'{count_product} / {len(products)} Elemento escrapeado')

        current_product={'market':'Coto','category': categorias,
                         'currency':'ARG'}
        time.sleep(1)

        #link product
        #link_product=product_description.find('a')['href'] #lo mando abajo
        
        #current_product['link_pagina'] = link_product
        #description product
        description= product_description.find('div',class_='descrip_full').text
        #description_product.append(description)
        current_product['descripcion'] = description

        #precio por unidad
        try:
            unidad = product_description.find('div',class_='price_regular_container').text
            #precio_contado.append(unidad)
            current_product['precio'] = unidad
        except AttributeError:
            unidad = product_description.find('span',class_='atg_store_newPrice').text
            #precio_contado.append(unidad)
            current_product['precio'] = unidad


        #descuento
        try:
            descuento = product_description.find('span',class_='text_price_discount').text
            print(descuento)
            #promocion.append(descuento)
            current_product['descuento'] =descuento
        except AttributeError:
            #promocion.append(None)
            current_product['descuento'] =None

        #precio descuento
        try:
            precio_final=product_description.find('span',class_='price_discount').text
            print(precio_final)
            #precio_descuento.append(precio_final)
            current_product['precio_descuento'] = precio_final
        except AttributeError:
            #precio_descuento.append(None)
            current_product['precio_descuento'] = None

        link_product=product_description.find('a')['href'] 
        current_product['link_pagina'] = link_product #mande abajo

        try:
            precio_unidad = product_description.find('span',class_='unit').text
            current_product['precio_unidad'] =precio_unidad
        except:
            current_product['precio_unidad'] = None     

        product_list.append(current_product)

    return product_list





    






In [48]:

def full_urls(partial_link,names_categories):
    #PATH= 'C:\Program Files (x86)\chromedriver.exe'
    PATH= "/usr/lib/chromium-browser/chromedriver"

    link_total = 'https://www.cotodigital3.com.ar'+partial_link

    s = Service(PATH)

    driver = webdriver.Chrome(service=s)
    #driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
    
    time.sleep(8)
    driver.get(link_total)
    time.sleep(1)
    driver.maximize_window()
    time.sleep(2)
    

    cont=1
    print(f'cantidad hojas {cont}')
    final_work=[] # list where i hold all the data
    data= take_product(names_categories,link_total)
    
    final_work.extend(data)


    while True:
            try:
                next = driver.find_element(By.LINK_TEXT,'Sig')
                time.sleep(2)
                next.click()
                pages= driver.current_url
                cont+=1
                print(f'cantidad hojas {cont}')
                data = take_product(names_categories,pages)
                
                final_work.extend(data)
                #total_urls.append(pages)
                
                time.sleep(4)
            except:
                 break
                 
                 
    sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
    
    for i in range(1,len(sig)):
        try:
            sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
            time.sleep(2)
            sig[i].click()
            cont+=1
            print(f'cantidad hojas {cont}')
            pages= driver.current_url
            data = take_product(names_categories,pages)
            
            final_work.extend(data)
            #total_urls.append(pages)
            time.sleep(4)
        except IndexError:
            sig = driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
            time.sleep(2)
            sig[1].click()
            cont+=1
            print(f'cantidad hojas {cont}')
            pages=driver.current_url
            data = take_product(names_categories,pages)
            
            final_work.extend(data)
            #total_urls.append(pages)
            time.sleep(4)
    
    
    

    driver.quit()

    print(f' the len of final_work is {len(final_work)}')

    df = pd.DataFrame(final_work)

    return df




In [67]:
data = full_urls('/sitios/cdigi/browse/catalogo-electro-informatica/_/N-u4pa0j','informatica')

cantidad hojas 1
categoria : informatica
1 / 12 Elemento escrapeado
categoria : informatica
2 / 12 Elemento escrapeado
1 Pago 15%
$14449
categoria : informatica
3 / 12 Elemento escrapeado
1 Pago 15%
$2719
categoria : informatica
4 / 12 Elemento escrapeado
categoria : informatica
5 / 12 Elemento escrapeado
1 Pago 15%
$42499
categoria : informatica
6 / 12 Elemento escrapeado
1 Pago 15%
$1784
categoria : informatica
7 / 12 Elemento escrapeado
1 Pago 15%
$1495
categoria : informatica
8 / 12 Elemento escrapeado
1 Pago 15%
$934
categoria : informatica
9 / 12 Elemento escrapeado
categoria : informatica
10 / 12 Elemento escrapeado
1 Pago 15%
$38249
categoria : informatica
11 / 12 Elemento escrapeado
categoria : informatica
12 / 12 Elemento escrapeado
cantidad hojas 2
categoria : informatica
1 / 12 Elemento escrapeado
1 Pago 15%
$22949
categoria : informatica
2 / 12 Elemento escrapeado
1 Pago 15%
$70549
categoria : informatica
3 / 12 Elemento escrapeado
categoria : informatica
4 / 12 Elemento e

In [49]:
links = ['/sitios/cdigi/browse/catalogo-almac%C3%A9n-arroz-y-legumbres/_/N-c0x2yz',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-especias/_/N-1t0tm80']

categoria =['arroz y legumbre', 'especias']




main_data=[]
#for link,title in zip(first_step[0],first_step[1]):

for link, title in zip (links,categoria):
    try:
        
        data1=full_urls(link,title)
        main_data.append(data1)
    except:
        print('#######################################################################')
        print("--------------------------NO SCRAPEO-----------------------------------")
        print('#######################################################################')
        pass

cantidad hojas 1
categoria : arroz y legumbre
1 / 12 Elemento escrapeado
categoria : arroz y legumbre
2 / 12 Elemento escrapeado
categoria : arroz y legumbre
3 / 12 Elemento escrapeado
categoria : arroz y legumbre
4 / 12 Elemento escrapeado
categoria : arroz y legumbre
5 / 12 Elemento escrapeado
categoria : arroz y legumbre
6 / 12 Elemento escrapeado
categoria : arroz y legumbre
7 / 12 Elemento escrapeado
categoria : arroz y legumbre
8 / 12 Elemento escrapeado
categoria : arroz y legumbre
9 / 12 Elemento escrapeado
categoria : arroz y legumbre
10 / 12 Elemento escrapeado
categoria : arroz y legumbre
11 / 12 Elemento escrapeado
categoria : arroz y legumbre
12 / 12 Elemento escrapeado
cantidad hojas 2
categoria : arroz y legumbre
1 / 12 Elemento escrapeado
categoria : arroz y legumbre
2 / 12 Elemento escrapeado
categoria : arroz y legumbre
3 / 12 Elemento escrapeado
categoria : arroz y legumbre
4 / 12 Elemento escrapeado
categoria : arroz y legumbre
5 / 12 Elemento escrapeado
categoria :

In [66]:
df

,market,category,currency,descripcion,precio,descuento,precio_descuento,link_pagina,precio_unidad
0,Coto,arroz y legumbre,ARG,Garbanzos COTO Bolsa 400 Gr,"$575,19...",None,None,/sitios/cdigi/producto/-garbanzos-coto-bolsa-4...,"Precio por 1 Kilo escurrido: $1.437,97"
1,Coto,arroz y legumbre,ARG,Maiz COTO Pisingallo Bolsa 400 Gr,"$341,05...",None,None,/sitios/cdigi/producto/-maiz-coto-pisingallo-b...,"Precio por 1 Kilo escurrido: $852,62"
2,Coto,arroz y legumbre,ARG,Porotos Alubia COTO Bolsa 500 Gr,"$788,83...",None,None,/sitios/cdigi/producto/-porotos-alubia-coto-bo...,"Precio por 1 Kilo escurrido: $1.577,66"
3,Coto,arroz y legumbre,ARG,Arvejas COTO Partidas Bolsa 400 Gr,"$414,40...",None,None,/sitios/cdigi/producto/-arvejas-coto-partidas-...,"Precio por 1 Kilo escurrido: $1.036,00"
4,Coto,arroz y legumbre,ARG,Lentejon COTO Bolsa 400 Gr,"$795,67...",None,None,/sitios/cdigi/producto/-lentejon-coto-bolsa-40...,"Precio por 1 Kilo escurrido: $1.989,17"
...,...,...,...,...,...,...,...,...,...
194,Coto,especias,ARG,Condimento Pimienta Negra Molida Badia 99.2 Grm,"$2.012,...",None,None,/sitios/cdigi/producto/-condimento-pimienta-ne...,"Precio por 1 Kilo : $20.292,03"
195,Coto,especias,ARG,Condimento Pimienta Roja Badia 113.4 Grm,"$1.506,...",None,None,/sitios/cdigi/producto/-condimento-pimienta-ro...,"Precio por 1 Kilo : $13.284,92"
196,Coto,especias,ARG,Comino Molido Alicante Sob 25 Grm,"$168,98...",None,None,/sitios/cdigi/producto/-comino-molido-alicante...,"Precio por 100 Gramos : $675,92"
197,Coto,especias,ARG,Tomillo Dos Anclas 35 Grm,"$213,04...",None,None,/sitios/cdigi/producto/-tomillo-dos-anclas-35-...,"Precio por 100 Gramos : $608,69"


In [57]:
df=pd.concat(main_data)

    

In [62]:
#df['precio'] = df['precio'].str.lstrip()
#df['precio'] = df['precio'].str.rstrip()
df['precio'] =df['precio'].str.replace('\t','').str.replace('\n','')

#df['precio_unidad'] = df['precio_unidad'].str.lstrip()
#df['precio_unidad'] = df['precio_unidad'].str.rstrip()
df['precio_unidad'] =df['precio_unidad'].str.replace('\t','').str.replace('\n','').str.replace('  ','')

In [64]:
df['precio'].value_counts()

                                       $1.123,49         17
                                       $533,97            5
                                       $201,01            4
                                       $107,90            4
                                       $8.190,03          3
                                                         ..
                                       $375,11            1
                                       $829,00            1
                                       $3.791,00          1
                                       $465,97            1
                                       $426,81            1
Name: precio, Length: 267, dtype: int64

In [59]:
df['precio_unidad']

0      Precio por 1 Kilo escurrido:                  ...
1      Precio por 1 Kilo escurrido:                  ...
2      Precio por 1 Kilo escurrido:                  ...
3      Precio por 1 Kilo escurrido:                  ...
4      Precio por 1 Kilo escurrido:                  ...
                             ...                        
194    Precio por 1 Kilo :                       $20....
195    Precio por 1 Kilo :                       $13....
196    Precio por 100 Gramos :                       ...
197    Precio por 100 Gramos :                       ...
198    Precio por 100 Gramos :                       ...
Name: precio_unidad, Length: 309, dtype: object